In [1]:
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torchvision
import math
from torch.autograd import Variable
import copy
import torchvision.transforms as transforms

In [3]:
import pandas as pd 
import numpy as np

In [ ]:
#not using bours of restaurants, since we have no information on what time people are looking to eat

In [4]:
rating_dir = './RCdata/rating_final.csv'
rest_profile_dir = './RCdata/geoplaces2.csv'
rest_cuisine_dir = './RCdata/chefmozcuisine.csv'
rest_payment_dir = './RCdata/chefmozaccepts.csv'
rest_parking_dir = './RCdata/chefmozparking.csv'
user_cuisine_dir = './RCdata/usercuisine.csv'
user_payment_dir = './RCdata/userpayment.csv'
user_profile_dir = './RCdata/userprofile.csv'

In [5]:
rest_profile = pd.read_csv(rest_profile_dir, encoding = 'cp1252')
rest_cuisine = pd.read_csv(rest_cuisine_dir) #encoding = 'cp1252')
rest_payment = pd.read_csv(rest_payment_dir)
rest_parking = pd.read_csv(rest_parking_dir)

In [ ]:
#setting up the data into one pandas dataframe for user attributes and one pandas dataframe for restaurant attributes

In [6]:
rest_parking.columns

Index(['placeID', 'parking_lot'], dtype='object')

In [7]:
n_rests = len(rest_profile.placeID)
print (n_rests)

130


In [8]:
rests = np.array(rest_profile.placeID)

In [9]:
cuisine_data = [0 for i in range(n_rests)]

In [10]:
for i in range(len(cuisine_data)):
    cuisines = rest_cuisine[rest_cuisine['placeID'] == rests[i]]['Rcuisine']
    this_one = ''
    if len(cuisines)>0: 
        for j in range(len(cuisines)):
            this_one = this_one + cuisines.iloc[j] + ' '        
    cuisine_data[i] = this_one

In [11]:
rest_profile.loc[:,'Cuisine'] = pd.Series(cuisine_data, index=rest_profile.index)

In [12]:
payment_data = [0 for i in range(n_rests)]

In [13]:
for i in range(len(payment_data)):
    payment = rest_payment[rest_payment['placeID'] == rests[i]]['Rpayment']
    this_one = ''
    if len(payment)>0: 
        for j in range(len(payment)):
            this_one = this_one + payment.iloc[j] + ' '        
    payment_data[i] = this_one

In [14]:
rest_profile.loc[:,'Payment'] = pd.Series(payment_data, index=rest_profile.index)

In [15]:
parking_data = [0 for i in range(n_rests)]

In [16]:
for i in range(len(parking_data)):
    parking = rest_parking[rest_parking['placeID'] == rests[i]]['parking_lot']
    this_one = ''
    if len(parking)>0: 
        for j in range(len(parking)):
            this_one = this_one + parking.iloc[j] + ' '        
    parking_data[i] = this_one

In [17]:
rest_profile.loc[:,'Parking'] = pd.Series(parking_data, index=rest_profile.index)

In [18]:
rest_profile.shape

(130, 24)

In [19]:
rest_profile.columns

Index(['placeID', 'latitude', 'longitude', 'the_geom_meter', 'name', 'address',
       'city', 'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
       'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
       'area', 'other_services', 'Cuisine', 'Payment', 'Parking'],
      dtype='object')

In [20]:
user_profile = pd.read_csv(user_profile_dir)
user_payment = pd.read_csv(user_payment_dir)
user_cuisine = pd.read_csv(user_cuisine_dir)

In [21]:
user_cuisine.columns

Index(['userID', 'Rcuisine'], dtype='object')

In [22]:
n_users = len(user_profile.userID)
print (n_users)

138


In [23]:
users = np.array(user_profile.userID)

In [24]:
cuisine_data = [0 for i in range(n_users)]

In [25]:
for i in range(len(cuisine_data)):
    cuisines = user_cuisine[user_cuisine['userID'] == users[i]]['Rcuisine']
    this_one = ''
    if len(cuisines)>0: 
        for j in range(len(cuisines)):
            this_one = this_one + cuisines.iloc[j] + ' '        
    cuisine_data[i] = this_one

In [26]:
user_profile.loc[:,'Cuisine'] = pd.Series(cuisine_data, index=user_profile.index)

In [27]:
payment_data = [0 for i in range(n_users)]

In [28]:
for i in range(len(payment_data)):
    payment = user_payment[user_payment['userID'] == users[i]]['Upayment']
    this_one = ''
    if len(payment)>0: 
        for j in range(len(payment)):
            this_one = this_one + payment.iloc[j] + ' '        
    payment_data[i] = this_one

In [29]:
user_profile.loc[:,'Payment'] = pd.Series(payment_data, index=user_profile.index)

In [31]:
rest_profile

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,accessibility,price,url,Rambience,franchise,area,other_services,Cuisine,Payment,Parking
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none,Japanese,,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,completely,low,?,familiar,f,open,none,Mexican,cash,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,partially,medium,?,familiar,f,open,none,Mexican,cash VISA MasterCard-Eurocard,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,completely,low,?,familiar,t,closed,none,Armenian,cash,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,completely,medium,?,familiar,t,closed,none,Mexican,cash,yes
5,135040,22.135617,-100.969709,0101000020957F00001B552189B84A58C15A2AAEFD2CA2...,Restaurant los Compadres,Camino a Simon Diaz 155 Centro,San Luis Potosi,SLP,Mexico,?,...,no_accessibility,high,?,familiar,f,closed,none,,cash VISA MasterCard-Eurocard,yes
6,132732,23.754357,-99.171288,0101000020957F00008A20E615808157C16272FECBF84F...,Taqueria EL amigo,Calle Mezquite Fracc Framboyanes,Cd Victoria,Tamaulipas,Mexico,?,...,completely,low,?,familiar,f,open,none,Mexican,cash,none
7,132875,22.149901,-100.993779,0101000020957F00008A2A0747DE4758C11EB31D2A31A8...,shi ro ie,?,?,?,?,?,...,no_accessibility,high,?,familiar,t,open,Internet,Japanese,cash VISA MasterCard-Eurocard American_Express...,valet parking
8,132609,23.760268,-99.165865,0101000020957F0000A478418BBA8057C133851EB22C4E...,Pollo_Frito_Buenos_Aires,tampico,victoria,Tamaulipas,Mexico,?,...,completely,low,?,quiet,t,closed,none,Fast_Food,cash,yes
9,135082,22.151448,-100.915099,0101000020957F0000A29FAF95CD4958C1FEEEBB73A991...,la Estrella de Dimas,Villa de Pozos 192 Villa de Pozos,San Luis Potosi,SLP,Mexico,?,...,no_accessibility,medium,?,familiar,f,closed,none,,cash,none


In [30]:
user_profile

,userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,...,interest,personality,religion,activity,color,weight,budget,height,Cuisine,Payment
0,U1001,22.139997,-100.978803,false,abstemious,informal,family,on foot,single,independent,...,variety,thrifty-protector,none,student,black,69,medium,1.77,American,cash
1,U1002,22.150087,-100.983325,false,abstemious,informal,family,public,single,independent,...,technology,hunter-ostentatious,Catholic,student,red,40,low,1.87,Mexican,cash
2,U1003,22.119847,-100.946527,false,social drinker,formal,family,public,single,independent,...,none,hard-worker,Catholic,student,blue,60,low,1.69,Mexican,cash
3,U1004,18.867000,-99.183000,false,abstemious,informal,family,public,single,independent,...,variety,hard-worker,none,professional,green,44,medium,1.53,Bakery Breakfast-Brunch Japanese Contemporary ...,cash bank_debit_cards
4,U1005,22.183477,-100.959891,false,abstemious,no preference,family,public,single,independent,...,none,thrifty-protector,Catholic,student,black,65,medium,1.69,American,cash
5,U1006,22.150000,-100.983000,true,social drinker,no preference,friends,car owner,single,independent,...,variety,hard-worker,none,student,blue,75,medium,1.80,Mexican,cash
6,U1007,22.118464,-100.938256,false,casual drinker,informal,solitary,public,single,independent,...,variety,thrifty-protector,Catholic,student,purple,60,low,1.59,Family,cash
7,U1008,22.122989,-100.923811,false,social drinker,formal,solitary,public,single,independent,...,technology,hard-worker,Catholic,student,green,68,low,1.72,Cafe-Coffee_Shop Mexican Juice Hawaiian Hot_Do...,cash
8,U1009,22.159427,-100.990448,false,abstemious,formal,family,on foot,single,kids,...,variety,thrifty-protector,Catholic,student,green,75,medium,1.78,Diner Fast_Food Family Cafe-Coffee_Shop Deli-S...,cash
9,U1010,22.190889,-100.998669,false,social drinker,no preference,friends,car owner,married,kids,...,technology,hard-worker,none,student,green,40,medium,1.67,Mexican,cash


In [32]:
user_profile.columns

Index(['userID', 'latitude', 'longitude', 'smoker', 'drink_level',
       'dress_preference', 'ambience', 'transport', 'marital_status', 'hijos',
       'birth_year', 'interest', 'personality', 'religion', 'activity',
       'color', 'weight', 'budget', 'height', 'Cuisine', 'Payment'],
      dtype='object')

In [33]:
rest_profile.columns

Index(['placeID', 'latitude', 'longitude', 'the_geom_meter', 'name', 'address',
       'city', 'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
       'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
       'area', 'other_services', 'Cuisine', 'Payment', 'Parking'],
      dtype='object')

In [ ]:
#Ratings start

In [34]:
ratings = pd.read_csv(rating_dir)

In [35]:
from sklearn.model_selection import train_test_split
rating_train, rating_test = train_test_split(ratings, test_size=0.3)    

In [36]:
n_users = len(ratings.userID.unique())
n_items = len(ratings.placeID.unique())
print (n_users)
print (n_items)

138
130


In [37]:
user_list = np.array(ratings.userID.unique())
item_list = np.array(ratings.placeID.unique())

In [ ]:
#build the context matrix 

In [38]:
def get_context(user, rest, user_profile, rest_profile):
    this_user = user_profile[user_profile['userID']==user]
    this_rest = rest_profile[rest_profile['placeID'] == rest]
    
    #print (this_user)
    #print (this_rest)
    contexts = 0
    
    if this_user['smoker'].item() == 'true' and (this_rest['smoking_area'].item() == 'section'or this_rest['smoking_area'].item() == 'permitted' or this_rest['smoking_area'].item() == 'only at bar'):
        contexts = contexts + 1
    
    if (this_user['drink_level'].item() == 'casual drinker' or this_user['drink_level'].item() == 'social_drinker') and this_rest['alcohol'].item() == 'Wine-Beer' or this_rest['alcohol'].item() == 'Full_Bar':
        contexts = contexts + 1 
    
    if (this_user['dress_preference'].item() == 'formal' or this_user['dress_preference'].item() == 'elegant') and (this_rest['dress_code'].item() == 'formal' or this_rest['Rambience'].item() == 'quiet' or this_rest['franchise'].item()== 'f'):
        contexts = contexts + 1 
        
    if (this_user['dress_preference'].item() == 'informal') and (this_rest['dress_code'].item() == 'informal' or this_rest['dress_code'].item() == 'casual'):
        contexts = contexts + 1 
    
    if (this_user['ambience'].item() == 'family' or this_user['ambience'].item() == 'friends') and (this_rest['Rambience'].item() == 'familiar'):
        contexts = contexts + 1 
    
    if (this_user['ambience'].item() == 'solitary') and (this_rest['Rambience'].item() == 'quiet'):
        contexts = contexts + 1
    
    if (this_user['transport'].item() == 'car owner') and (this_rest['Parking'].item() == 'yes' or this_rest['Parking'].item() == 'public' or this_rest['Parking'].item() == 'valet parking'):
        contexts = contexts + 1 
        
    if (this_user['activity'].item() == 'student' or this_user['activity'].item() == 'working class') and (this_rest['price'].item() == 'low' or this_rest['price'].item() == 'medium'):
        contexts = contexts + 1 
    
    if (this_user['activity'].item() == 'unemployed') and (this_rest['price'].item() == 'low'):
        contexts = contexts + 1 
    
    if (this_user['activity'].item() == 'professional') and (this_rest['price'].item() == 'medium' or this_rest['price'].item() == 'high'):
        contexts = contexts + 1 
    
    if (this_user['budget'].item() == 'high') and (this_rest['price'].item() == 'high'):
        contexts = contexts + 1 
    
    if (this_user['budget'].item() == 'medium') and (this_rest['price'].item() == 'medium'):
        contexts = contexts + 1 
    
    if (this_user['budget'].item() == 'low') and (this_rest['price'].item() == 'low' or this_rest['franchise'].item() == 't'):
        contexts + contexts + 1 
        
    this_user_cuisines = this_user['Cuisine'].item().split(' ')
    this_rest_cuisines = this_rest['Cuisine'].item()
    
    this_user_payment = this_user['Payment'].item().split(' ')
    this_rest_payment = this_rest['Payment'].item()
    
    for j in range(len(this_user_cuisines)-1):
        if this_user_cuisines[j] in this_rest_cuisines:
            contexts = contexts + 1
    
    for m in range(len(this_user_payment)-1):
        if this_user_payment[m] in this_rest_payment:
            contexts = contexts + 1 
        
    return contexts
      

In [39]:
from scipy.sparse import rand as sprand
from scipy.sparse import lil_matrix

In [ ]:
#do we need to create contexts for the entirety of the restaurants? 

In [40]:
contexts_full = lil_matrix((n_users, n_items), dtype = float)
for i in range(len(user_list)):
    for j in range(len(item_list)):
        contexts_full[i, j] = get_context(user_list[i], item_list[j], user_profile, rest_profile)

In [41]:
context_dense = contexts_full.todense()

In [42]:
np.max(context_dense)

11.0

In [43]:
np.mean(context_dense)

3.6567447045707917

In [44]:
contexts_norm = lil_matrix((n_users, n_items), dtype = float)
for i in range(len(user_list)):
    for j in range(len(item_list)):
        contexts_norm[i, j] = (get_context(user_list[i], item_list[j], user_profile, rest_profile))/11.0

In [45]:
interactions_full_orig = lil_matrix((n_users, n_items), dtype = float)
for row in ratings.itertuples():
    user, = np.where(user_list == row[1])
    item, = np.where(item_list == row[2])
    interactions_full_orig[user[0], item[0]] = row[3] 

In [46]:
interactions_train_orig = lil_matrix((n_users, n_items), dtype = float)
context_train = lil_matrix((n_users, n_items), dtype = float)
for row in rating_train.itertuples():
    user, = np.where(user_list == row[1])
    item, = np.where(item_list == row[2])
    context_train[user[0], item[0]] = get_context(row[1], row[2], user_profile, rest_profile)
    interactions_train_orig[user[0], item[0]] = row[3]

In [47]:
#original ratings
interactions_test_orig = lil_matrix((n_users, n_items), dtype = float)
context_test = lil_matrix((n_users, n_items), dtype = float)
for row in rating_test.itertuples():
    user, = np.where(user_list == row[1])
    item, = np.where(item_list == row[2])
    interactions_test_orig[user[0], item[0]] = row[3]
    context_test[user[0], item[0]] = get_context(row[1], row[2], user_profile, rest_profile)

In [48]:
class MatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items, n_factors=5):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users,
                                               n_factors,
                                               sparse=False)
        self.item_factors = torch.nn.Embedding(n_items,n_factors,sparse=False)
                  
    
    def forward(self, users, items, context):
        #return (self.user_factors(user) * self.item_factors(item)).sum(1)
    
        orig = torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))
        
        #here = torch.from_numpy(context)
        #context = torch.mm(self.user_context(users), torch.transpose(self.item_context(items),0,1))
        
        #fused_context = context * self.fusion.expand_as(context)
        
        mat = torch.add(orig, context)
        return mat
        #return torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))
    

In [49]:
class BiasedMatrixFactorization(torch.nn.Module):
    
    def __init__(self, n_users, n_items,mean, n_factors=5):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users,
                                               n_factors,
                                               sparse=False)
        self.item_factors = torch.nn.Embedding(n_items, n_factors,sparse=False)
        self.user_biases = torch.nn.Embedding(n_users, 1, sparse = False)
        
        self.item_biases = torch.nn.Embedding(n_items,1, sparse = False)
                                               
        self.mu = mean
    
    
    def forward(self, users, items, context):
        
        item_means = []
        user_means = []
        for i in range(len(users)):
            item_means.append(torch.t(self.item_biases(items)))
            
        for j in range(len(items)):
            user_means.append(torch.t(self.user_biases(users)))
        
        it_means = torch.cat(item_means, 0)
        us_means = torch.cat(user_means, 0)
        us_means = torch.t(us_means)
        
        orig = torch.mm(self.user_factors(users),torch.transpose(self.item_factors(items),0,1))
        
        y = torch.add(orig, it_means)
        
        f = torch.add(y,us_means)
        
        result = torch.add(f,self.mu)
        
        mat = torch.add(result, context)
        
        return mat

In [50]:
def get_batch(batch_size,ratings):
    # Sort our data and scramble it
    rows, cols = ratings.shape
    p = np.random.permutation(rows)
    
    # create batches
    sindex = 0
    eindex = batch_size
    while eindex < rows:
        batch = p[sindex:eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch

    if eindex >= rows:
        batch = range(sindex,rows)
        yield batch

In [51]:
def load_model(model,load_path):
    load_dict = torch.load(load_path)
    val_loss = load_dict['val_loss']
    model.load_state_dict(load_dict['model_state_dict'])


def checkpoint_model(val_loss, model,save_path):
    save_dict = dict(
                     val_loss=val_loss,
                     model_state_dict=model.state_dict())
                     #opt_state_dict=predictions.state_dict())
    torch.save(save_dict, save_path)

In [52]:
def run_validation(model, test_ratings, context_mat, fusion, BATCH_SIZE, load_path):
    load_model(model, load_path)
    running_loss = 0.0
    loss_func = torch.nn.MSELoss()
    accur = 0
    for i,batch in enumerate(get_batch(BATCH_SIZE, test_ratings)):
        
        interactions = Variable(torch.FloatTensor(test_ratings[batch, :].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(test_ratings.shape[1])))
        
        first = (context_mat[batch,:].toarray())*fusion
        contexts = Variable(torch.FloatTensor(first))
        
        
        predictions = model(rows, cols, contexts)
        loss = loss_func(predictions, interactions)
        
        running_loss += np.sqrt(loss.data[0])*BATCH_SIZE
        
        correct = get_accuracy_test(test_ratings, model, rows, fusion, context_mat)
        accur = accur+ correct
    
    epoch_loss = running_loss/test_ratings.shape[0]
    return epoch_loss,accur

In [53]:
def run_validation_test(model, test_ratings, test_ratings_orig, BATCH_SIZE, load_path):
    load_model(model, load_path)
    running_loss = 0.0
    loss_func = torch.nn.MSELoss()
    accur = 0
    for i,batch in enumerate(get_batch(BATCH_SIZE, test_ratings)):
        
        interactions = Variable(torch.FloatTensor(test_ratings[batch, :].toarray()))
        rows = Variable(torch.LongTensor(batch))
        cols = Variable(torch.LongTensor(np.arange(test_ratings.shape[1])))
        
        predictions = model(rows, cols)
        loss = loss_func(predictions, interactions)
        
        running_loss += np.sqrt(loss.data[0])*BATCH_SIZE
        
        correct = get_accuracy_test(test_ratings_orig, model, rows)
        accur = accur+ correct
    
    epoch_loss = running_loss/test_ratings.shape[0]
    return epoch_loss,accur

In [54]:
def get_accuracy_test(interactions_orig,model,rows,fusion, contexts_full):
    
    checks = np.nonzero(interactions_orig)
    check_these = checks[0]
    
    batch = range(interactions_orig.shape[0])
    interactions = Variable(torch.FloatTensor(interactions_orig[batch,:].toarray()))
    users = Variable(torch.LongTensor(batch))
    cols = Variable(torch.LongTensor(np.arange(interactions_orig.shape[1])))
    
    first = (contexts_full[batch,:].toarray())*fusion
    contexts = Variable(torch.FloatTensor(first))
    
    predictions = model(users, cols, contexts)
    
    x = predictions.data.numpy()
    
    correct = 0
    
    for u in range(len(rows)):
        user = rows.data[u]
        where = np.where(check_these==user)
        index = where[0]
        if len(index) > 0:
            for c in range(len(index)):
                rest = checks[1][index[c]]
                prediction = predictions[u][rest]
                   
                val = prediction.data[0]
                
                if val <= np.percentile(x,22):
                    this_val = 0.0
                elif np.percentile(x,22) < val < np.percentile(x, 58):
                    this_val = 1.0
                elif val >= np.percentile(x, 58):
                     this_val= 2.0
        
                
                if this_val == interactions_orig[user,rest]:
                    correct = correct + 1 
            else:
                next
    return correct

In [55]:
def generate_predictions(model, contexts_full, interactions_full, load_path):
    load_model(model, load_path)

    batch = range(interactions_full.shape[0])
    interactions = Variable(torch.FloatTensor(interactions_full[batch,:].toarray()))
    rows = Variable(torch.LongTensor(batch))
    cols = Variable(torch.LongTensor(np.arange(interactions_full.shape[1])))
    first = (contexts_full[batch,:].toarray())*fusion
        
    #contexts = Variable(torch.FloatTensor(contexts_full[batch,:].toarray()))
    contexts = Variable(torch.FloatTensor(first))
        
    predictions = model(rows, cols, contexts)
    
    predictions[interactions!=0] = 0
        
    val, index = torch.sort(predictions,1, descending = True)
        
    top5 = index[:,:5]
        
    return top5

In [56]:
def give_top5_cuisines(recs, name_list, cuisine_list, user_list, rest_list):
    your_rests = [[0 for j in range(11)] for u in range(len(user_list))]
    for i in range(len(user_list)):
        user, = np.where(user_list == user_list[i])
        your_rests[i][0] = user_list[i]
        count = 1
        for r in range(5):
            this_rest = recs[user[0]][r].data[0]
            rest = rest_list[this_rest]
            x = name_list.loc[name_list['placeID'] == rest]
            name = x['name'].item()
            
            your_rests[i][count] = name
            count = count +1
            
            y = cuisine_list.loc[cuisine_list['placeID'] == rest]['Rcuisine']
            
            cuisine_types = ''
            for m in y:
                cuisine_types = cuisine_types + m + ', '
            
            your_rests[i][count] = cuisine_types
            count = count + 1
            
    return your_rests

In [57]:
def give_number_of_contexts(recs, context, this_user,user_list, rest_list):
   
    contexts = [0 for i in range(5)]
    
    user, = np.where(this_user == user_list)
    print (user)
    
    for u in range(5):
        this_rest = recs[user[0]][u].data[0]
        contexts[u] = context[user[0], this_rest]
    
    return contexts

In [58]:
batch_size = 10
save_path = './project_models/model.ckpt'
save_path_val = './project_models/modelv.ckpt'
factors = [2,3,4,5]
fusion_arr = [0.0,0.25,0.5, 0.75, 1.0]

In [59]:
checks = np.nonzero(interactions_train_orig)
checks_v = np.nonzero(interactions_test_orig)

In [60]:
#original ratings
best_val_loss = 1000.0
best_val_accuracy = 0.0
#FOCUS ON FACTOR 2
for x in range(1):
    print ('Factor ' + str(factors[x]))
    for f in range(len(fusion_arr)):  
        fusion = fusion_arr[f]
        print ('FUSION: ' + str(fusion))
        best_train_loss = 1000.0
        #accur = 0
        model = MatrixFactorization(interactions_train_orig.shape[0], interactions_train_orig.shape[1], n_factors=factors[x])
        loss_func = torch.nn.MSELoss()
        reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-6, weight_decay=0.001)
        for i in range(5):
            print ("Epoch: " + str(i))
            running_loss = 0.0
            accur = 0
            #best_train_loss = 1000.0
            for m, batch in enumerate(get_batch(batch_size, interactions_train_orig)):
                reg_loss_func.zero_grad()
                interactions = Variable(torch.FloatTensor(interactions_train_orig[batch,:].toarray()))
                rows = Variable(torch.LongTensor(batch))
                cols = Variable(torch.LongTensor(np.arange(interactions_train_orig.shape[1])))
                first = (contexts_full[batch,:].toarray())*fusion
        
                #contexts = Variable(torch.FloatTensor(contexts_full[batch,:].toarray()))
                contexts = Variable(torch.FloatTensor(first))
        
                predictions = model(rows, cols, contexts)
        
                loss = loss_func(predictions, interactions)
                running_loss += np.sqrt(loss.data[0])*batch_size
                
                #correct = get_accuracy_test(interactions_train_orig, model, rows, fusion, contexts_full)
                correct = get_accuracy_test(interactions_train_orig, model, rows, fusion, contexts_full)
                accur = accur + correct
                
                loss.backward()
            reg_loss_func.step()
    
            epoch_loss = running_loss/interactions_train_orig.shape[0]
            print ('train_loss: ' + str(epoch_loss))
            
            print ('accuracy:' + str(accur/len(checks[0])))
            
            if epoch_loss < best_train_loss:
                best_train_loss = epoch_loss
                print ('checkpoint model with train loss: ' + str(best_train_loss))
                checkpoint_model(epoch_loss, model, save_path)
        
            val_loss, val_accuracy = run_validation(model,interactions_test_orig, contexts_full,fusion, batch_size, save_path)
            #val_loss, val_accuracy = run_validation(model,interactions_test_orig, contexts_full, fusion, batch_size, save_path)
            print ('val loss: ' + str(val_loss))
            val_percent = val_accuracy/len(checks_v[0])
            print ('val accuracy: ' + str(val_accuracy/len(checks_v[0])))
        
        
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                print ('checkpointing_model with val loss: ' + str(best_val_loss))
                #checkpoint_model(val_loss, model, save_path_val)
            if val_percent > best_val_accuracy:
                best_val_accuracy = val_percent
                checkpoint_model(val_loss, model, save_path_val)
                print ('New Best Val Accuracy: ' + str(best_val_accuracy) + ' with fusion: ' + str(fusion) + ' and factor :' + str(factors[x]))

Factor 2
FUSION: 0.0
Epoch: 0
train_loss: 1.2899284662815858
accuracy:0.38193343898573695
checkpoint model with train loss: 1.2899284662815858
val loss: 1.2433671076665689
val accuracy: 0.39855072463768115
checkpointing_model with val loss: 1.2433671076665689
New Best Val Accuracy: 0.39855072463768115 with fusion: 0.0 and factor :2
Epoch: 1
train_loss: 1.3010751876010456
accuracy:0.39461172741679873
val loss: 1.273206456379087
val accuracy: 0.34782608695652173
Epoch: 2
train_loss: 1.2777367199939635
accuracy:0.3692551505546751
checkpoint model with train loss: 1.2777367199939635
val loss: 1.2686425494435392
val accuracy: 0.3442028985507246
Epoch: 3
train_loss: 1.3058623348935663
accuracy:0.37083993660855785
val loss: 1.28512146352817
val accuracy: 0.35144927536231885
Epoch: 4
train_loss: 1.2932747913423777
accuracy:0.36450079239302696
val loss: 1.270082911529183
val accuracy: 0.35507246376811596
FUSION: 0.25
Epoch: 0
train_loss: 1.7846971932917868
accuracy:0.4294770206022187
checkpoint

In [61]:
recs = generate_predictions(model, contexts_full, interactions_full_orig, save_path_val)
results = give_top5_cuisines(recs, rest_profile, rest_cuisine, user_list, item_list)

In [62]:
give_number_of_contexts(recs, contexts_full, 'U1093', user_list,item_list)

[18]


[7.0, 6.0, 7.0, 6.0, 6.0]

In [63]:
give_number_of_contexts(recs, contexts_full, 'U1077', user_list,item_list)

[0]


[9.0, 7.0, 8.0, 7.0, 7.0]

In [64]:
col_names = ['User','Rec1', 'Rec1_Cuisines', 'Rec2', 'Rec2_Cuisines', 'Rec3', 'Rec3_Cuisines', 'Rec4','Rec4_Cuisines',  'Rec5', 'Rec5_Cuisines']

In [65]:
df = pd.DataFrame(results,columns = col_names)

In [66]:
df

,User,Rec1,Rec1_Cuisines,Rec2,Rec2_Cuisines,Rec3,Rec3_Cuisines,Rec4,Rec4_Cuisines,Rec5,Rec5_Cuisines
0,U1077,El Oceano Dorado,"Mexican,",Restaurante la Cantina,"Bar, Bar_Pub_Brewery,",Gordas de morales,"Mexican,",La Virreina,"Mexican,",Restaurant Orizatlan,"Mexican,"
1,U1068,La Virreina,"Mexican,",Gordas de morales,"Mexican,",Restaurante la Cantina,"Bar, Bar_Pub_Brewery,",Restaurant Orizatlan,"Mexican,",Restaurante de Mariscos la Langosta,"Seafood,"
2,U1067,Cenaduria El RincÃ³n de Tlaquepaque,"Mexican,",El Rincon de San Francisco,"Mexican,",el pueblito,"Mexican,",vips,"Mexican,",Log Yin,"Mexican, Mediterranean, Burgers,"
3,U1103,El Oceano Dorado,"Mexican,",Gordas de morales,"Mexican,",La Estrella de Dimas,"Mexican,",La Virreina,"Mexican,",Restaurante la Cantina,"Bar, Bar_Pub_Brewery,"
4,U1107,Restaurante y Pescaderia Tampico,"Seafood,",TACOS CORRECAMINOS,"Mexican,",La Estrella de Dimas,"Mexican,",La Posada del Virrey,"International,",Gordas de morales,"Mexican,"
5,U1044,El Oceano Dorado,"Mexican,",Restaurante la Cantina,"Bar, Bar_Pub_Brewery,",Gordas de morales,"Mexican,",La Virreina,"Mexican,",vips,"Mexican,"
6,U1070,el pueblito,"Mexican,",El Rincón de San Francisco,"Mexican,",Cenaduria El RincÃ³n de Tlaquepaque,"Mexican,",El cotorreo,"Family, Mexican,",El Rincon de San Francisco,"Mexican,"
7,U1031,Log Yin,"Mexican, Mediterranean, Burgers,",tacos los volcanes,"American,",Gorditas Dona Tota,"Mexican,",La Estrella de Dimas,"Mexican,",Restaurante El Cielo Potosino,
8,U1082,Gordas de morales,"Mexican,",Restaurante la Cantina,"Bar, Bar_Pub_Brewery,",carnitas_mata,"Mexican,",La Virreina,"Mexican,",Dominos Pizza,"Pizzeria,"
9,U1023,Log Yin,"Mexican, Mediterranean, Burgers,",Gorditas Dona Tota,"Mexican,",Restaurante El Cielo Potosino,,Cenaduria El RincÃ³n de Tlaquepaque,"Mexican,",el pueblito,"Mexican,"


In [517]:
len(df['Rec5'].value_counts().index.tolist())

55

In [440]:
np.array(user_profile[user_profile['userID']=='U1093'])

array([['U1093', 18.871654, -99.183251, 'true', 'casual drinker',
        'no preference', 'family', 'car owner', 'single', 'independent',
        1990, 'variety', 'thrifty-protector', 'Catholic', 'student',
        'blue', 80, 'medium', 1.67, 'Japanese ',
        'American_Express VISA ']], dtype=object)

In [67]:
np.array(user_profile[user_profile['userID']=='U1077'])

array([['U1077', 22.156469, -100.98554, 'false', 'social drinker',
        'elegant', 'family', 'public', 'married', 'kids', 1987,
        'technology', 'thrifty-protector', 'Catholic', 'student', 'blue',
        65, 'medium', 1.71, 'Mexican ', 'VISA cash bank_debit_cards ']],
      dtype=object)

In [68]:
#normalized context matrix
best_val_loss = 1000.0
best_val_accuracy = 0.0
#FOCUS ON FACTOR 2
for x in range(1):
    print ('Factor ' + str(factors[x]))
    for f in range(len(fusion_arr)):  
        fusion = fusion_arr[f]
        print ('FUSION: ' + str(fusion))
        best_train_loss = 1000.0
        #accur = 0
        model = MatrixFactorization(interactions_train_orig.shape[0], interactions_train_orig.shape[1], n_factors=factors[x])
        loss_func = torch.nn.MSELoss()
        reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-6, weight_decay=0.001)
        for i in range(5):
            print ("Epoch: " + str(i))
            running_loss = 0.0
            accur = 0
            #best_train_loss = 1000.0
            for m, batch in enumerate(get_batch(batch_size, interactions_train_orig)):
                reg_loss_func.zero_grad()
                interactions = Variable(torch.FloatTensor(interactions_train_orig[batch,:].toarray()))
                rows = Variable(torch.LongTensor(batch))
                cols = Variable(torch.LongTensor(np.arange(interactions_train_orig.shape[1])))
                first = (contexts_norm[batch,:].toarray())*fusion
        
                #contexts = Variable(torch.FloatTensor(contexts_full[batch,:].toarray()))
                contexts = Variable(torch.FloatTensor(first))
        
                predictions = model(rows, cols, contexts)
        
                loss = loss_func(predictions, interactions)
                running_loss += np.sqrt(loss.data[0])*batch_size
                
                #correct = get_accuracy_test(interactions_train_orig, model, rows, fusion, contexts_full)
                correct = get_accuracy_test(interactions_train_orig, model, rows, fusion, contexts_norm)
                accur = accur + correct
                
                loss.backward()
            reg_loss_func.step()
    
            epoch_loss = running_loss/interactions_train_orig.shape[0]
            print ('train_loss: ' + str(epoch_loss))
            
            print ('accuracy:' + str(accur/len(checks[0])))
            
            if epoch_loss < best_train_loss:
                best_train_loss = epoch_loss
                print ('checkpoint model with train loss: ' + str(best_train_loss))
                checkpoint_model(epoch_loss, model, save_path)
        
            val_loss, val_accuracy = run_validation(model,interactions_test_orig, contexts_norm,fusion, batch_size, save_path)
        #val_loss, val_accuracy = run_validation(model,interactions_test_orig, contexts_full, fusion, batch_size, save_path)
            print ('val loss: ' + str(val_loss))
            val_percent = val_accuracy/len(checks_v[0])
            print ('val accuracy: ' + str(val_accuracy/len(checks_v[0])))
        
        
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                print ('checkpointing_model with val loss: ' + str(best_val_loss))
                #checkpoint_model(val_loss, model, save_path_val)
            if val_percent > best_val_accuracy:
                best_val_accuracy = val_percent
                checkpoint_model(val_loss, model, save_path_val)
                print ('New Best Val Accuracy: ' + str(best_val_accuracy) + ' with fusion: ' + str(fusion) + ' and factor :' + str(factors[x]))

Factor 2
FUSION: 0.0
Epoch: 0
train_loss: 1.434099882752502
accuracy:0.3993660855784469
checkpoint model with train loss: 1.434099882752502
val loss: 1.438129933440376
val accuracy: 0.4420289855072464
checkpointing_model with val loss: 1.438129933440376
New Best Val Accuracy: 0.4420289855072464 with fusion: 0.0 and factor :2
Epoch: 1
train_loss: 1.4356869178372171
accuracy:0.42789223454833597
val loss: 1.3761599139903582
val accuracy: 0.427536231884058
checkpointing_model with val loss: 1.3761599139903582
Epoch: 2
train_loss: 1.4293264789283984
accuracy:0.3993660855784469
checkpoint model with train loss: 1.4293264789283984
val loss: 1.4211824804498998
val accuracy: 0.4782608695652174
New Best Val Accuracy: 0.4782608695652174 with fusion: 0.0 and factor :2
Epoch: 3
train_loss: 1.420900422111574
accuracy:0.3993660855784469
checkpoint model with train loss: 1.420900422111574
val loss: 1.4225963705280082
val accuracy: 0.4057971014492754
Epoch: 4
train_loss: 1.4425453120531373
accuracy:0.3

In [74]:
mean = np.mean(interactions_train_orig)
save_path_b = './project_models/bmodel.ckpt'
save_path_val_b = './project_models/bmodelv.ckpt'

In [75]:
#original ratings
best_val_loss = 1000.0
best_val_accuracy = 0.0
#FOCUS ON FACTOR 2
for x in range(1):
    print ('Factor ' + str(factors[x]))
    for f in range(len(fusion_arr)):  
        fusion = fusion_arr[f]
        print ('FUSION: ' + str(fusion))
        best_train_loss = 1000.0
        #accur = 0
        model = BiasedMatrixFactorization(interactions_train_orig.shape[0], interactions_train_orig.shape[1],mean, n_factors=factors[x])
        loss_func = torch.nn.MSELoss()
        reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-6, weight_decay=0.001)
        for i in range(5):
            print ("Epoch: " + str(i))
            running_loss = 0.0
            accur = 0
            #best_train_loss = 1000.0
            for m, batch in enumerate(get_batch(batch_size, interactions_train_orig)):
                reg_loss_func.zero_grad()
                interactions = Variable(torch.FloatTensor(interactions_train_orig[batch,:].toarray()))
                rows = Variable(torch.LongTensor(batch))
                cols = Variable(torch.LongTensor(np.arange(interactions_train_orig.shape[1])))
                first = (contexts_full[batch,:].toarray())*fusion
        
                #contexts = Variable(torch.FloatTensor(contexts_full[batch,:].toarray()))
                contexts = Variable(torch.FloatTensor(first))
        
                predictions = model(rows, cols, contexts)
        
                loss = loss_func(predictions, interactions)
                running_loss += np.sqrt(loss.data[0])*batch_size
                
                #correct = get_accuracy_test(interactions_train_orig, model, rows, fusion, contexts_full)
                correct = get_accuracy_test(interactions_train_orig, model, rows, fusion, contexts_full)
                accur = accur + correct
                
                loss.backward()
            reg_loss_func.step()
    
            epoch_loss = running_loss/interactions_train_orig.shape[0]
            print ('train_loss: ' + str(epoch_loss))
            
            print ('accuracy:' + str(accur/len(checks[0])))
            
            if epoch_loss < best_train_loss:
                best_train_loss = epoch_loss
                print ('checkpoint model with train loss: ' + str(best_train_loss))
                checkpoint_model(epoch_loss, model, save_path_b)
        
            val_loss, val_accuracy = run_validation(model,interactions_test_orig, contexts_full,fusion, batch_size, save_path_b)
            #val_loss, val_accuracy = run_validation(model,interactions_test_orig, contexts_full, fusion, batch_size, save_path)
            print ('val loss: ' + str(val_loss))
            val_percent = val_accuracy/len(checks_v[0])
            print ('val accuracy: ' + str(val_accuracy/len(checks_v[0])))
        
        
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                print ('checkpointing_model with val loss: ' + str(best_val_loss))
                checkpoint_model(val_loss, model, save_path_val_b)
            if val_percent > best_val_accuracy:
                best_val_accuracy = val_percent
                print ('New Best Val Accuracy: ' + str(best_val_accuracy) + ' with fusion: ' + str(fusion) + ' and factor :' + str(factors[x]))

Factor 2
FUSION: 0.0
Epoch: 0
train_loss: 2.206165300390469
accuracy:0.35974643423137875
checkpoint model with train loss: 2.206165300390469
val loss: 2.176410438107027
val accuracy: 0.3442028985507246
checkpointing_model with val loss: 2.176410438107027
New Best Val Accuracy: 0.3442028985507246 with fusion: 0.0 and factor :2
Epoch: 1
train_loss: 2.194751315569511
accuracy:0.375594294770206
checkpoint model with train loss: 2.194751315569511
val loss: 2.1858352789692934
val accuracy: 0.40217391304347827
New Best Val Accuracy: 0.40217391304347827 with fusion: 0.0 and factor :2
Epoch: 2
train_loss: 2.226424293848455
accuracy:0.3866877971473851
val loss: 2.184090392139076
val accuracy: 0.4492753623188406
New Best Val Accuracy: 0.4492753623188406 with fusion: 0.0 and factor :2
Epoch: 3
train_loss: 2.2092163952759396
accuracy:0.40887480190174325
val loss: 2.1770620257252244
val accuracy: 0.37318840579710144
Epoch: 4
train_loss: 2.2080222408629844
accuracy:0.40729001584786056
val loss: 2.183

In [76]:
recs = generate_predictions(model, contexts_full, interactions_full_orig, save_path_val_b)
results = give_top5_cuisines(recs, rest_profile, rest_cuisine, user_list, item_list)

In [77]:
df = pd.DataFrame(results,columns = col_names)

In [78]:
df

,User,Rec1,Rec1_Cuisines,Rec2,Rec2_Cuisines,Rec3,Rec3_Cuisines,Rec4,Rec4_Cuisines,Rec5,Rec5_Cuisines
0,U1077,El Oceano Dorado,"Mexican,",El Rincon de San Francisco,"Mexican,",Gordas de morales,"Mexican,",La Estrella de Dimas,"Mexican,",Restaurant Orizatlan,"Mexican,"
1,U1068,El Rincon de San Francisco,"Mexican,",La Cantina Restaurante,"Bar, Bar_Pub_Brewery,",shi ro ie,"Japanese,",Restaurant and Bar and Clothesline Carlos N Ch...,"Bar, Bar_Pub_Brewery,",Abondance Restaurante Bar,"Bar,"
2,U1067,El Rincon de San Francisco,"Mexican,",La Cantina Restaurante,"Bar, Bar_Pub_Brewery,",Cenaduria El RincÃ³n de Tlaquepaque,"Mexican,",Gordas de morales,"Mexican,",tortas hawai,"Fast_Food,"
3,U1103,La Cantina Restaurante,"Bar, Bar_Pub_Brewery,",El Rincon de San Francisco,"Mexican,",Restaurante la Estrella de Dima,,El Mundo de la Pasta,"Italian,",Cenaduria El RincÃ³n de Tlaquepaque,"Mexican,"
4,U1107,La Cantina Restaurante,"Bar, Bar_Pub_Brewery,",shi ro ie,"Japanese,",El Rincon de San Francisco,"Mexican,",Restaurant and Bar and Clothesline Carlos N Ch...,"Bar, Bar_Pub_Brewery,",Koye Sushi,"Chinese,"
5,U1044,El Oceano Dorado,"Mexican,",Gordas de morales,"Mexican,",Restaurant Orizatlan,"Mexican,",Rincon del Bife,,la parroquia,"Breakfast-Brunch,"
6,U1070,El cotorreo,"Family, Mexican,",Chilis Cuernavaca,,La Virreina,"Mexican,",Restaurant Bar Hacienda los Martinez,"Bar,",Cenaduria El RincÃ³n de Tlaquepaque,"Mexican,"
7,U1031,Restaurante la Cantina,"Bar, Bar_Pub_Brewery,",La Virreina,"Mexican,",El Mundo de la Pasta,"Italian,",El Rincon de San Francisco,"Mexican,",Restaurante la Estrella de Dima,
8,U1082,El Rincon de San Francisco,"Mexican,",Gordas de morales,"Mexican,",El Oceano Dorado,"Mexican,",Restaurant Orizatlan,"Mexican,",Restaurant and Bar and Clothesline Carlos N Ch...,"Bar, Bar_Pub_Brewery,"
9,U1023,El Rincon de San Francisco,"Mexican,",shi ro ie,"Japanese,",La Cantina Restaurante,"Bar, Bar_Pub_Brewery,",Restaurant Orizatlan,"Mexican,",Gordas de morales,"Mexican,"


In [79]:
give_number_of_contexts(recs, contexts_full, 'U1093', user_list,item_list)

[18]


[7.0, 6.0, 5.0, 4.0, 7.0]

In [80]:
give_number_of_contexts(recs, contexts_full, 'U1077', user_list,item_list)

[0]


[9.0, 7.0, 8.0, 8.0, 7.0]

In [487]:
len(df['Rec5'].value_counts().index.tolist())

51

In [81]:
#normalized context
best_val_loss = 1000.0
best_val_accuracy = 0.0
#focus on factor 2
for x in range(1):
    print ('Factor ' + str(factors[x]))
    for f in range(len(fusion_arr)):  
        fusion = fusion_arr[f]
        print ('FUSION: ' + str(fusion))
        best_train_loss = 1000.0
        #accur = 0
        model = BiasedMatrixFactorization(interactions_train_orig.shape[0], interactions_train_orig.shape[1], mean, n_factors=factors[x])
        loss_func = torch.nn.MSELoss()
        reg_loss_func = torch.optim.Adam(model.parameters(), lr=1e-6, weight_decay=0.001)
        for i in range(5):
            print ("Epoch: " + str(i))
            running_loss = 0.0
            accur = 0
            #best_train_loss = 1000.0
            for m, batch in enumerate(get_batch(batch_size, interactions_train_orig)):
                reg_loss_func.zero_grad()
                interactions = Variable(torch.FloatTensor(interactions_train_orig[batch,:].toarray()))
                rows = Variable(torch.LongTensor(batch))
                cols = Variable(torch.LongTensor(np.arange(interactions_train_orig.shape[1])))
                first = (contexts_norm[batch,:].toarray())*fusion
        
                #contexts = Variable(torch.FloatTensor(contexts_full[batch,:].toarray()))
                contexts = Variable(torch.FloatTensor(first))
        
                predictions = model(rows, cols, contexts)
        
                loss = loss_func(predictions, interactions)
                running_loss += np.sqrt(loss.data[0])*batch_size
                
                #correct = get_accuracy_test(interactions_train_orig, model, rows, fusion, contexts_full)
                correct = get_accuracy_test(interactions_train_orig, model, rows, fusion, contexts_norm)
                accur = accur + correct
                
                loss.backward()
            reg_loss_func.step()
    
            epoch_loss = running_loss/interactions_train_orig.shape[0]
            print ('train_loss: ' + str(epoch_loss))
            
            print ('accuracy:' + str(accur/len(checks[0])))
            
            if epoch_loss < best_train_loss:
                best_train_loss = epoch_loss
                print ('checkpoint model with train loss: ' + str(best_train_loss))
                checkpoint_model(epoch_loss, model, save_path_b)
        
            val_loss, val_accuracy = run_validation(model,interactions_test_orig, contexts_norm,fusion, batch_size, save_path_b)
            #val_loss, val_accuracy = run_validation(model,interactions_test_orig, contexts_full, fusion, batch_size, save_path)
            print ('val loss: ' + str(val_loss))
            val_percent = val_accuracy/len(checks_v[0])
            print ('val accuracy: ' + str(val_accuracy/len(checks_v[0])))
        
        
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                print ('checkpointing_model with val loss: ' + str(best_val_loss))
                checkpoint_model(val_loss, model, save_path_val_b)
            if val_percent > best_val_accuracy:
                best_val_accuracy = val_percent
                print ('New Best Val Accuracy: ' + str(best_val_accuracy) + ' with fusion: ' + str(fusion) + ' and factor :' + str(factors[x]))

Factor 2
FUSION: 0.0
Epoch: 0
train_loss: 2.0892900094666085
accuracy:0.3676703645007924
checkpoint model with train loss: 2.0892900094666085
val loss: 2.0817479820737184
val accuracy: 0.427536231884058
checkpointing_model with val loss: 2.0817479820737184
New Best Val Accuracy: 0.427536231884058 with fusion: 0.0 and factor :2
Epoch: 1
train_loss: 2.0839961437806807
accuracy:0.3803486529318542
checkpoint model with train loss: 2.0839961437806807
val loss: 2.0872434085808385
val accuracy: 0.3442028985507246
Epoch: 2
train_loss: 2.103097561669648
accuracy:0.39461172741679873
val loss: 2.0802194065297477
val accuracy: 0.3695652173913043
checkpointing_model with val loss: 2.0802194065297477
Epoch: 3
train_loss: 2.07106260114798
accuracy:0.4041204437400951
checkpoint model with train loss: 2.07106260114798
val loss: 2.083855482056931
val accuracy: 0.38768115942028986
Epoch: 4
train_loss: 2.075954574388206
accuracy:0.3898573692551506
val loss: 2.046349565579277
val accuracy: 0.37318840579710